In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_8eb7e5fdaa4240cba62b017a11bd6579(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'a163084eea8d49a9a8f6bfa978c7c788')
    hconf.set(prefix + '.username', 'c2569e0540c04f3a891153e1bde7444b')
    hconf.set(prefix + '.password', 'yrf=1Bf^u!doiP8b')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_8eb7e5fdaa4240cba62b017a11bd6579(name)


In [2]:
genome_tags_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://DefaultProjectalemreoz95gmailcom." + name + "/genome-tags.csv")

In [3]:
tags_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://DefaultProjectalemreoz95gmailcom." + name + "/tags.csv")

In [4]:
genome_scores_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://DefaultProjectalemreoz95gmailcom." + name + "/genome-scores.csv")

In [5]:
df = tags_df.join(genome_tags_df,"tag","inner").select("userId","movieId","tagId")

In [6]:
df2 = df.join(genome_scores_df,["movieId","tagId"],"inner")

In [7]:
from pyspark.sql.functions import *
final_df = df2.select("userId","movieId","tagId",(tanh((((df2.relevance)*20)-12))).alias("relevance"))

In [8]:
final_df.registerTempTable("tempDF")

In [9]:
sqlContext.cacheTable("tempDF")

In [10]:
finalDFrame = sqlContext.sql("SELECT userId, movieId, tagId, relevance FROM tempDF ORDER BY userId, movieId, tagId")

In [11]:
top1000ReliableUser = finalDFrame.select("userId","relevance").groupBy("userId").sum("relevance").sort("sum(relevance)",ascending=False).limit(1000)

In [12]:
top1000ReliableUser.show(1000)

+------+------------------+
|userId|    sum(relevance)|
+------+------------------+
| 88738| 5886.228605226696|
| 58612|1700.3142105607253|
| 52814|1648.8382288692467|
| 11081|1582.5985838545296|
| 10616| 1447.994398848874|
|  9815|1413.2447981826163|
| 68558|1261.1889719076676|
|  1741| 1119.254178487935|
| 25737|1068.4360438182061|
|  4450|1000.3835964912714|
| 27898| 969.4690500707228|
|122523| 961.2716671956211|
|119367| 959.6825546999671|
| 77463| 927.4315434523476|
|124998| 854.3481069572272|
| 77297| 845.1531603150678|
| 28906| 840.9469449647919|
|120937| 797.6761602947428|
|  6431| 665.8821765083227|
|111982| 662.9396577253003|
|132195| 662.4140364061684|
| 10514| 639.5480101354963|
|107711| 607.9753558248359|
|131900| 591.0842834020642|
| 20845| 577.0503608958547|
| 28780| 575.6579425139815|
| 57434| 566.7686207532322|
|130446| 553.6307449052149|
| 72766| 544.8235867476017|
| 49129| 544.6906064447293|
| 68625| 535.7112386908598|
| 42245|  531.450439524062|
| 96370| 528.4794209

In [13]:
ratings_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://DefaultProjectalemreoz95gmailcom." + name + "/ratings.csv")

In [14]:
df3 = top1000ReliableUser.join(ratings_df,"userId","inner")

In [15]:
temp_total_df = top1000ReliableUser.join(ratings_df,"userId","inner")

In [16]:
temp_best_movie_df = temp_total_df.select(temp_total_df["movieId"],temp_total_df["rating"] * temp_total_df["sum(relevance)"])

In [17]:
bestMovies = temp_best_movie_df.select("movieId","(rating * sum(relevance))").groupBy("movieId").sum("(rating * sum(relevance))").sort("sum((rating * sum(relevance)))",ascending=False).limit(20)

In [18]:
movies_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://DefaultProjectalemreoz95gmailcom." + name + "/movies.csv")

In [19]:
tempBestMovieDF = bestMovies.join(movies_df,"movieId","inner")

In [20]:
import pyspark.sql.functions
best20Movies = tempBestMovieDF.select("title","sum((rating * sum(relevance)))").sort(desc("sum((rating * sum(relevance)))"))

In [21]:
best20Movies.show()

+--------------------+------------------------------+
|               title|sum((rating * sum(relevance)))|
+--------------------+------------------------------+
|  Matrix, The (1999)|             471354.5594368036|
|   Fight Club (1999)|             426152.0577531984|
| Pulp Fiction (1994)|            415367.08530751534|
|Shawshank Redempt...|             409525.1721352335|
| Forrest Gump (1994)|             395281.5232354895|
|Lord of the Rings...|             395052.4140476215|
|      Memento (2000)|            378664.83073142625|
|    Inception (2010)|             375745.9660923454|
|Lord of the Rings...|             372978.4285739495|
|Lord of the Rings...|            364419.68578229164|
|Dark Knight, The ...|            361753.23284971487|
|Silence of the La...|             360313.6505073855|
|Star Wars: Episod...|             359782.0039532395|
|American Beauty (...|             358955.4184849295|
|Back to the Futur...|            352814.61000777036|
|Eternal Sunshine ...|      